In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# --- Load Data ---
df = pd.read_csv("iris.csv")

# Independent (X) and Dependent (y) variables
X = df.iloc[:, 0:4].values     # sepal length, sepal width, petal length, petal width
y = df.iloc[:, 4].values       # species

# --- Preprocessing ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode target labels
encoder = OneHotEncoder(sparse_output=False)  # use sparse=False if older sklearn
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# --- Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=18
)

# --- Activation functions ---
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    return x * (1 - x)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / np.sum(e_x, axis=1, keepdims=True)

# --- Loss function ---
def cross_entropy(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-9), axis=1))

# --- Network Parameters ---
input_size = 4
hidden_size = 5
output_size = 3
lr = 0.01
epochs = 10000

# --- Initialize Weights ---
w1 = np.random.randn(input_size, hidden_size)
b1 = np.zeros((1, hidden_size))
w2 = np.random.randn(hidden_size, output_size)
b2 = np.zeros((1, output_size))

# --- Training Loop ---
for epoch in range(epochs):
    # Forward pass
    Z1 = X_train @ w1 + b1
    A1 = sigmoid(Z1)
    Z2 = A1 @ w2 + b2
    A2 = softmax(Z2)

    # Compute loss
    loss = cross_entropy(y_train, A2)

    # Backpropagation
    dZ2 = A2 - y_train
    dw2 = A1.T @ dZ2
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    dA1 = dZ2 @ w2.T
    dZ1 = dA1 * sigmoid_deriv(A1)
    dw1 = X_train.T @ dZ1
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    # Update weights
    w1 -= lr * dw1
    w2 -= lr * dw2
    b1 -= lr * db1
    b2 -= lr * db2

    # Print every 100 epochs
    if epoch % 1000 == 0:
        print(f"Epoch {epoch} | Loss: {loss:.4f}")

Epoch 0 | Loss: 1.7478
Epoch 1000 | Loss: 0.0505
Epoch 2000 | Loss: 0.0483
Epoch 3000 | Loss: 0.0462
Epoch 4000 | Loss: 0.0434
Epoch 5000 | Loss: 0.0402
Epoch 6000 | Loss: 0.0372
Epoch 7000 | Loss: 0.0348
Epoch 8000 | Loss: 0.0330
Epoch 9000 | Loss: 0.0316


In [27]:
# --- Testing ---
Z1_test = X_test @ w1 + b1
A1_test = sigmoid(Z1_test)
Z2_test = A1_test @ w2 + b2
A2_test = softmax(Z2_test)

# Convert one-hot to labels
y_pred = np.argmax(A2_test, axis=1)
y_true = np.argmax(y_test, axis=1)

# --- Evaluation ---
print("\n✅ Accuracy:", accuracy_score(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=encoder.categories_[0]))


✅ Accuracy: 1.0

Confusion Matrix:
 [[12  0  0]
 [ 0  8  0]
 [ 0  0 10]]

Classification Report:
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        12
Iris-versicolor       1.00      1.00      1.00         8
 Iris-virginica       1.00      1.00      1.00        10

       accuracy                           1.00        30
      macro avg       1.00      1.00      1.00        30
   weighted avg       1.00      1.00      1.00        30

